In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!git clone https://github.com/valkryhx/chatGLM-6B-QLoRA

fatal: destination path 'chatGLM-6B-QLoRA' already exists and is not an empty directory.


In [2]:
!git pull --all --force
#!pip install peft==0.4.0
#!pip install  -U git+https://github.com/huggingface/peft.git
%cd /kaggle/working/chatGLM-6B-QLoRA 
!ls
!pip install -r requirements.txt
#!pip install deepspeed==0.9.5  这个也是需要的 但是目前kaggle 的runtime自带了

fatal: not a git repository (or any parent up to mount point /kaggle)
Stopping at filesystem boundary (GIT_DISCOVERY_ACROSS_FILESYSTEM not set).
/kaggle/working/chatGLM-6B-QLoRA
README.md
chatGLM_6B_QLoRA.json
data
ds_zero2_config.json
ds_zero3_config.json
inference_qlora.py
inference_test.py
luzi.json
merge_lora_and_quantize.py
output-history-sft-0803-v1
output-sharegpt-sft-0802-v1
output-sharegpt-sft-0803-2kdata-v4
output-sharegpt-sft-0803-v1
output-sharegpt-sft-0803-v2
output-sharegpt-sft-0803-v3
pics
pretrain_chatglm2_v5.ipynb
pretrain_chatglm2_v7.ipynb
pretrain_chatglm2_v8.ipynb
pretrain_qlora_chatglm2.py
qa_aug.json
qlora_param.json
remote_scripts
requirements.txt
sft_multi_turn_qlora_chatglm2.py
sharegpt_sft_success_0802.ipynb
state.db
train_ds_zero2_test.py——错误的
train_normal_lora.py
train_qlora.py
train_qlora_deepspeed_zero.py
二次预训练pt_sft_继续lora训练_qlora_glm_成功0730.ipynb
参考与经验


## <font color= brown>SFT sharegpt格式的多轮对话数据</font>

In [ ]:
--resume_from_checkpoint ./output-sharegpt-sft-0803-v2/checkpoint-310 \

In [26]:
# ./data/sharegpt_multi_turn_data 目录不能有空白json文件
!git pull --all --force 
#!pip install  -U git+https://github.com/huggingface/peft.git   # 20230717 peft==0.4.0正式发布了 不用调版本了推理完后再训练需要重新升级到0.4.0dev 所以有这句
!deepspeed --include localhost:0,1  sft_multi_turn_qlora_chatglm2.py  \
  --train_args_json luzi.json \
  --model_name_or_path THUDM/chatglm2-6b \
  --output_dir output-sharegpt-sft-0803-medi-v5 \
  --num_train_samples -1 \
  --num_eval_samples 20 \
  --train_data_path ./data/medi  \
  --eval_data_path  ./data/medi    \
  --data_type sharegpt  \
  --max_length 1800 \
  --lora_rank 64 \
  --lora_dropout 0.05 \
  --compute_dtype fp16 \
  --per_device_train_batch_size 2 \
  --per_device_eval_batch_size 2  \
  --gradient_accumulation_steps 1 \
  --learning_rate  1e-4 \
  --num_train_epochs  40  \
  --save_total_limit 2 \
  --load_in_4bit True \
--deepspeed ds_zero2_config.json

Fetching origin
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 2), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), 773 bytes | 386.00 KiB/s, done.
From https://github.com/valkryhx/chatGLM-6B-QLoRA
   8527711..cf76749  main       -> origin/main
Updating 8527711..cf76749
Fast-forward
 sft_multi_turn_qlora_chatglm2.py | 5 ++++-
 1 file changed, 4 insertions(+), 1 deletion(-)
[2023-08-03 16:06:08,165] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)
/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins

In [12]:
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm2-6b", trust_remote_code=True)
input_ids = tokenizer(
            "1+2+3+4",
            return_tensors="pt",
            padding="max_length",
            max_length=20,
            truncation=True,
            add_special_tokens=False
        ).input_ids

In [15]:
input_ids.ne(0).int()

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1]],
       dtype=torch.int32)

In [18]:
a=[1,2,3,4,1]
b=torch.Tensor(a)
b

tensor([1., 2., 3., 4., 1.])

In [20]:
b.ne(1).int()

tensor([0, 1, 1, 1, 0], dtype=torch.int32)

In [ ]:
model.model.attentions.query_key_value.layer

# sharegpt 格式数据sft后推理测试

In [ ]:
!git pull --all --force
#!pip install peft==0.4.0
#!pip install  -U git+https://github.com/huggingface/peft.git
%cd /kaggle/working/chatGLM-6B-QLoRA 
!ls
!pip install -r requirements.txt
#!pip install deepspeed==0.9.5  这个也是需要的 但是目前kaggle 的runtime自带了

In [10]:
import os
import argparse
from typing import List, Dict, Optional
from accelerate import init_empty_weights  # load an empty model,just structure , no real weight.
import bitsandbytes as bnb
import torch
from glob import glob
from loguru import logger
from datasets import load_dataset
from transformers import (
    AutoModel,
    AutoModelForCausalLM,
    AutoTokenizer,
    HfArgumentParser,
    set_seed,
    TrainingArguments,
    Trainer,
    BitsAndBytesConfig 
)
from peft import (
    TaskType,
    LoraConfig,
    #AdaLoraConfig ,  #  提出自2020年 感觉和lora区别不大 而且和qlora有冲突 这里代码没有用到 
                     #例子https://www.zhihu.com/question/596950521/answer/3109759716
    get_peft_model,
    set_peft_model_state_dict,
    prepare_model_for_kbit_training
)
from peft.utils import TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING
from transformers.deepspeed import HfDeepSpeedConfig
import deepspeed
import json
from itertools import chain

[2023-08-03 15:03:52,910] [INFO] [real_accelerator.py:110:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118.so
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib'), PosixPath('/usr/local/cuda/lib'), PosixPath('/usr/local/lib/x86_64-linux-gnu')}
  warn(msg)


In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel, PeftConfig


# 有效peft_model_path = '/kaggle/working/chatGLM-6B-QLoRA/output-sft-qlora-ds-yunguan0712-aug-v3'

peft_model_path= "/kaggle/working/chatGLM-6B-QLoRA/output-sharegpt-sft-0803-2kdata-v4/checkpoint-220"
config = PeftConfig.from_pretrained(peft_model_path)
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=torch.float16)
# base_model加载时保持和train时完全一致的参数配置
model = AutoModel.from_pretrained(config.base_model_name_or_path,
                                       quantization_config=q_config,
                                       trust_remote_code=True,
                                       device_map='auto')

tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path, trust_remote_code=True)
# input_text = '请你作为烽火SSE智能助手，帮我回答下面的问题。新型全电发票目前系统无法识别，要如何录入？'
# print(f'输入：\n{input_text}')
#tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path, trust_remote_code=True)

# response, history = base_model.chat(tokenizer=tokenizer,history=[], query=input_text)
# print(f'微调前：\n{response}')

# model = PeftModel.from_pretrained(base_model, peft_model_path)
# response, history = model.chat(tokenizer=tokenizer, history=[] ,query=input_text)
# print(f'微调后: \n{response}')

In [ ]:
# 不加载peft 基座模型推理
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path, trust_remote_code=True)

print("用户：你好，智能小助手\n")
old_response = ""
for response, history in model.stream_chat(tokenizer, "你好", [], max_length=1024, 
                                           top_p=0.9, temperature=0.75,
                                           num_beams=5):
    print(response[len(old_response):], end="")
    old_response = response
print(end="\r")
#print(1111,old_response)作为烽火智能助手，我遇到了以下故障现象，某地市某网络单盘占用槽位显示异常现场实际占用两个槽位。请给出类似案例以供参考。
#print("ChatGLM2-6B：\n",response)
#print("\n------------------------------------------------\n用户：")
#raise ValueError(123)
line = input()
past_key_values=None
history =[]
#prompt_format = "[Round {}]\n\n问：{}\n\n答："
while line:
    if line.strip()=="#QUIT": # 输入#QUIT退出对话循环
        break
    #inst = "我有关于烽火SSE的问题，请你作为烽火SSE智能助手，帮我回答下面的问题。" #计算两个矩阵的乘积："
    #line = PROMPT_TEMPLATE.format_map({'instruction': line})
    if line.strip()=="#CLEAR" :  #清空历史对话
        history=[]
    #format_line = prompt_format.format(1,line)
    #print(f"【formatted line】={format_line}")
    #old_response = ""
    beams =5
    print(f"num_beams={beams}")
    
    for response, history  in model.stream_chat(tokenizer=tokenizer, 
                                               history=history ,
                                               max_length=1024,
                                               query=line,
                                               top_p=0.95,temperature=0.55,
                                               num_beams=beams,
                                               return_past_key_values=False,
                                               past_key_values=None):
        #print(past_key_values) 不要打印 太长了
        print(response[len(old_response):], end="")
        old_response = response
    print(end="\r")
    #print(f"\nold response=\n{old_response}")
    #print(end="\r") # 量化后 num_beams只能=1 否则报错
    #print("ChatGLM2-6B：\n", old_response.replace("\\n",'\n'))
    #print("\n------------------------------------------------\n用户：")
    line = input()

In [ ]:
model = PeftModel.from_pretrained(model, peft_model_path)

#下面这三行貌似可以不用
#peft_model_dict = model.state_dict()
#peft_model_dict.update(torch.load(f"{peft_model_path}/adapter_model.bin"))
#model.load_state_dict(peft_model_dict,strict=False)

model.eval()

In [ ]:
我想开一辆移动餐车。我想做带有墨西哥风格影响的提升版希腊菜肴。你能为移动餐车写一个菜单吗？

In [ ]:
# 不加载peft 基座模型推理
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path, trust_remote_code=True)

print("用户：你好，智能小助手\n")
old_response = ""
for response, history in model.stream_chat(tokenizer, "你好", [], max_length=1024, 
                                           top_p=0.9, temperature=0.75,
                                           num_beams=5):
    print(response[len(old_response):], end="")
    old_response = response
print(end="\r")
#print(1111,old_response)作为烽火智能助手，我遇到了以下故障现象，某地市某网络单盘占用槽位显示异常现场实际占用两个槽位。请给出类似案例以供参考。
#print("ChatGLM2-6B：\n",response)
#print("\n------------------------------------------------\n用户：")
#raise ValueError(123)
line = input()
past_key_values=None
history =[]
#prompt_format = "[Round {}]\n\n问：{}\n\n答："
while line:
    if line.strip()=="#QUIT": # 输入#QUIT退出对话循环
        break
    #inst = "我有关于烽火SSE的问题，请你作为烽火SSE智能助手，帮我回答下面的问题。" #计算两个矩阵的乘积："
    #line = PROMPT_TEMPLATE.format_map({'instruction': line})
    if line.strip()=="#CLEAR" :  #清空历史对话
        history=[]
    #format_line = prompt_format.format(1,line)
    #print(f"【formatted line】={format_line}")
    #old_response = ""
    beams =5
    print(f"num_beams={beams}")
    
    for response, history  in model.stream_chat(tokenizer=tokenizer, 
                                               history=history ,
                                               max_length=1024,
                                               query=line,
                                               top_p=0.95,temperature=0.55,
                                               num_beams=beams,
                                               return_past_key_values=False,
                                               past_key_values=None):
        #print(past_key_values) 不要打印 太长了
        print(response[len(old_response):], end="")
        old_response = response
    print(end="\r")
    #print(f"\nold response=\n{old_response}")
    #print(end="\r") # 量化后 num_beams只能=1 否则报错
    #print("ChatGLM2-6B：\n", old_response.replace("\\n",'\n'))
    #print("\n------------------------------------------------\n用户：")
    line = input()

## <font color= brown>SFT 测试history格式的多轮对话数据 注意--data_type 为history</font>

In [ ]:
# ./data/sharegpt_multi_turn_data 目录不能有空白json文件
!git pull --all --force 
#!pip install  -U git+https://github.com/huggingface/peft.git   # 20230717 peft==0.4.0正式发布了 不用调版本了推理完后再训练需要重新升级到0.4.0dev 所以有这句
!deepspeed --include localhost:0,1  sft_multi_turn_qlora_chatglm2.py  \
  --train_args_json luzi.json \
  --model_name_or_path THUDM/chatglm2-6b \
  --output_dir output-history-sft-0803-v1 \
  --num_train_samples -1 \
  --num_eval_samples -1 \
  --train_data_path ./data/multi_turn_conversations/   \
  --eval_data_path  ./data/multi_turn_conversations    \
  --data_type history  \
  --max_length 1600 \
  --lora_rank 64 \
  --lora_dropout 0.05 \
  --compute_dtype fp16 \
  --per_device_train_batch_size 1 \
  --per_device_eval_batch_size 1  \
  --gradient_accumulation_steps 1 \
  --learning_rate  5e-5 \
  --num_train_epochs  40  \
  --save_total_limit 2 \
  --load_in_4bit True \
--deepspeed ds_zero2_config.json

# history格式数据sft后的推理测试

In [ ]:
import os
import argparse
from typing import List, Dict, Optional
from accelerate import init_empty_weights  # load an empty model,just structure , no real weight.
import bitsandbytes as bnb
import torch
from glob import glob
from loguru import logger
from datasets import load_dataset
from transformers import (
    AutoModel,
    AutoModelForCausalLM,
    AutoTokenizer,
    HfArgumentParser,
    set_seed,
    TrainingArguments,
    Trainer,
    BitsAndBytesConfig 
)
from peft import (
    TaskType,
    LoraConfig,
    #AdaLoraConfig ,  #  提出自2020年 感觉和lora区别不大 而且和qlora有冲突 这里代码没有用到 
                     #例子https://www.zhihu.com/question/596950521/answer/3109759716
    get_peft_model,
    set_peft_model_state_dict,
    prepare_model_for_kbit_training
)
from peft.utils import TRANSFORMERS_MODELS_TO_LORA_TARGET_MODULES_MAPPING
from transformers.deepspeed import HfDeepSpeedConfig
import deepspeed
import json
from itertools import chain

# 加载基座模型 注意qconfig

In [ ]:

import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel, PeftConfig


# 有效peft_model_path = '/kaggle/working/chatGLM-6B-QLoRA/output-sft-qlora-ds-yunguan0712-aug-v3'

peft_model_path= "./output-history-sft-0803-v1"
config = PeftConfig.from_pretrained(peft_model_path)
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=torch.float16)
# base_model加载时保持和train时完全一致的参数配置
base_model = AutoModel.from_pretrained(config.base_model_name_or_path,
                                       quantization_config=q_config,
                                       trust_remote_code=True,
                                       device_map='auto')

tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path, trust_remote_code=True)
# input_text = '请你作为烽火SSE智能助手，帮我回答下面的问题。新型全电发票目前系统无法识别，要如何录入？'
# print(f'输入：\n{input_text}')
#tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path, trust_remote_code=True)

# response, history = base_model.chat(tokenizer=tokenizer,history=[], query=input_text)
# print(f'微调前：\n{response}')

# model = PeftModel.from_pretrained(base_model, peft_model_path)
# response, history = model.chat(tokenizer=tokenizer, history=[] ,query=input_text)
# print(f'微调后: \n{response}')

In [ ]:
print("用户：你好，智能小助手\n")
old_response = ""
for response, history in base_model.stream_chat(tokenizer, "你好", [], max_length=1024, 
                                           top_p=0.7, temperature=0.95,
                                           num_beams=2):
    print(response[len(old_response):], end="")
    old_response = response
print(end="\r")
#print(1111,old_response)
#print("ChatGLM2-6B：\n",response)
#print("\n------------------------------------------------\n用户：")
#raise ValueError(123)
line = input()
while line:
    if line.strip()=="#QUIT": # 输入#QUIT退出对话循环
        break
    #inst = "我有关于烽火SSE的问题，请你作为烽火SSE智能助手，帮我回答下面的问题。" #计算两个矩阵的乘积："
    #line = PROMPT_TEMPLATE.format_map({'instruction': line})
    if line.strip()=="#CLEAR" :  #清空历史对话
        history=[]
    #line = inst  + line
    print(f"【formatted line】={line}")
    old_response = ""
    num_beams = 2
    print(f"num_beams={num_beams}")
    for response, history in base_model.stream_chat(tokenizer=tokenizer, 
                                               history=history ,
                                               max_length=1024,
                                               query=line,
                                               top_p=0.95,temperature=0.55,
                                               num_beams=num_beams):
        print(response[len(old_response):], end="")
        old_response = response
    print(end="\r")
    print(f"\nold response=\n{old_response}")
    print(end="\r") # 量化后 num_beams只能=1 否则报错
    print("ChatGLM2-6B：\n", old_response.replace("\\n",'\n'))
    print("\n------------------------------------------------\n用户：")
    line = input()

# 上面的测试中 使用基座模型 看得出 回答确实跟语料的内容没关系 毕竟没有经过私有语料训练
# 我们再看看经过训练后的模型回答

In [ ]:
peft_model = PeftModel.from_pretrained(base_model, peft_model_path)

#下面这三行貌似可以不用
#peft_model_dict = model.state_dict()
#peft_model_dict.update(torch.load(f"{peft_model_path}/adapter_model.bin"))
#model.load_state_dict(peft_model_dict,strict=False)

peft_model.eval()

# 查看一下layer weight
print(model.base_model.model.transformer.encoder.layers[27].self_attention.query_key_value.lora_A.default.weight)
print(model.base_model.model.transformer.encoder.layers[27].self_attention.query_key_value.lora_B.default.weight)

# qkv
print(model.base_model.model.transformer.encoder.layers[27].self_attention.query_key_value.weight)
# dense layer
print(model.base_model.model.transformer.encoder.layers[27].self_attention.dense.weight)
# core attention
print(model.base_model.model.transformer.encoder.layers[27].self_attention.core_attention)

In [ ]:
torch.cuda.empty_cache()

# 私有对话语料SFT后的推理 感觉好像因为语料太少 没学会什么东西

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path, trust_remote_code=True)

print("用户：你好，智能小助手\n")
old_response = ""
for response, history in peft_model.stream_chat(tokenizer, "你好", [], max_length=1024, 
                                           top_p=0.7, temperature=0.95,
                                           num_beams=5):
    print(response[len(old_response):], end="")
    old_response = response
print(end="\r")
#print(1111,old_response)作为烽火智能助手，我遇到了以下故障现象，某地市某网络单盘占用槽位显示异常现场实际占用两个槽位。请给出类似案例以供参考。
#print("ChatGLM2-6B：\n",response)
#print("\n------------------------------------------------\n用户：")
#raise ValueError(123)
line = input()
past_key_values=None
history =[]
prompt_format = "[Round {}]\n\n问：{}\n\n答："
while line:
    if line.strip()=="#QUIT": # 输入#QUIT退出对话循环
        break
    #inst = "我有关于烽火SSE的问题，请你作为烽火SSE智能助手，帮我回答下面的问题。" #计算两个矩阵的乘积："
    #line = PROMPT_TEMPLATE.format_map({'instruction': line})
    if line.strip()=="#CLEAR" :  #清空历史对话
        history=[]
    format_line = prompt_format.format(1,line)
    print(f"【formatted line】={format_line}")
    #old_response = ""
    beams =2
    print(f"num_beams={beams}")
    
    for response, history ,past_key_values in peft_model.stream_chat(tokenizer=tokenizer, 
                                               history=history ,
                                               max_length=512,
                                               query=format_line,
                                               top_p=0.95,temperature=0.75,
                                               num_beams=1,
                                               return_past_key_values=True,
                                               past_key_values=past_key_values):
        #print(past_key_values) 不要打印 太长了
        print(response[len(old_response):], end="")
        old_response = response
    print(end="\r")
    print(f"\nold response=\n{old_response}")
    print(end="\r") # 量化后 num_beams只能=1 否则报错
    print("ChatGLM2-6B：\n", old_response.replace("\\n",'\n'))
    print("\n------------------------------------------------\n用户：")
    line = input()

In [ ]:
prompt_format = "[Round {}]\n\n问：{}\n\n答："
line = "吃饭了吗"
prompt_format.format(1,line)